In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv("PPB_Affinity_processed.csv")

In [3]:
# these columns are created for processing purposes, they will be removed later
df["ligand"] = df["Ligand Sequences"].str.replace(",", "")
df["receptor"] = df["Receptor Sequences"].str.replace(",", "")
df["combined"] = df["ligand"] + df["receptor"]

In [4]:
df.head()

,Source Data Set,Complex ID,PDB,Mutations,Ligand Chains,Receptor Chains,Ligand Name,Receptor Name,KD(M),Affinity Method,...,PDB PubMed ID,PDB Release Date,Affinity PubMed ID,Affinity Release Date,Subgroup,Ligand Sequences,Receptor Sequences,ligand,receptor,combined
0,SKEMPI v2.0,"1A22:A, B::PMID=7504735",1A22,NaN,A,B,Human growth hormone,hGH binding protein,9.000000e-10,SPR,...,9571026.0,1998-04-29,7504735,1993 Dec 5,NaN,FPTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...,PKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQEWTQ...,FPTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...,PKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQEWTQ...,FPTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...
1,SKEMPI v2.0,"1A4Y:A, B::PMID=9050852",1A4Y,NaN,A,B,Ribonuclease inhibitor,Angiogenin,5.000000e-16,Other,...,9311977.0,1998-10-14,9050852,1997 Mar 4,NaN,SLDIQSLDIQCEELSDARWAELLPLLQQCQVVRLDDCGLTEARCKD...,QDNSRYTHFLTQHYDAKPQGRDDRYCESIMRRRGLTSPCKDINTFI...,SLDIQSLDIQCEELSDARWAELLPLLQQCQVVRLDDCGLTEARCKD...,QDNSRYTHFLTQHYDAKPQGRDDRYCESIMRRRGLTSPCKDINTFI...,SLDIQSLDIQCEELSDARWAELLPLLQQCQVVRLDDCGLTEARCKD...
2,SKEMPI v2.0,"1ACB:E, I::PMID=9048543",1ACB,NaN,E,I,Bovine alpha-chymotrypsin,Eglin c,1.490000e-12,IASP,...,1583684.0,1993-10-31,9048543,1997 Feb 18,NaN,CGVPAIQPVLSGLSRIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSL...,KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTLDLRYNRVR...,CGVPAIQPVLSGLSRIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSL...,KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTLDLRYNRVR...,CGVPAIQPVLSGLSRIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSL...
3,SKEMPI v2.0,"1AHW:A, B, C::PMID=9480775",1AHW,NaN,"A, B",C,Immunoglobulin fab 5G9,Tissue factor,3.400000e-09,IASP,...,9480775.0,1998-02-25,9480775,1998 Feb 6,NaN,DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKT...,TNTVAAYNLTWKSTNFKTILEWEPKPVNQVYTVQISTKSGDWKSKC...,DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKT...,TNTVAAYNLTWKSTNFKTILEWEPKPVNQVYTVQISTKSGDWKSKC...,DIKMTQSPSSMYASLGERVTITCKASQDIRKYLNWYQQKPWKSPKT...
4,SKEMPI v2.0,"1AK4:A, D::PMID=9223641",1AK4,NaN,A,D,Cyclophilin A,HIV-1 capsid protein,1.200000e-05,SPR,...,8980234.0,1997-10-15,9223641,1997 Jun 27,NaN,MVNPTVFFDIAVDGEPLGRVSFELFADKVPKTAENFRALSTGEKGF...,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...,MVNPTVFFDIAVDGEPLGRVSFELFADKVPKTAENFRALSTGEKGF...,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...,MVNPTVFFDIAVDGEPLGRVSFELFADKVPKTAENFRALSTGEKGF...


In [5]:
# convert KD to pKD
df["pKd"] = -np.log10(df["KD(M)"])
df["pKd"].describe()

count    12048.000000
mean         7.541897
std          2.144316
min          1.318759
25%          5.872895
50%          7.602060
75%          9.020015
max         15.698970
Name: pKd, dtype: float64

In [ ]:
# filter out rows where any ligand or receptor chain is shorter than 20
# amino acids
minlen = 20
f = lambda seqs: any([len(seq) < minlen for seq in seqs])
min_len_mask = df["Ligand Sequences"].str.split(",").apply(f) | df[
    "Receptor Sequences"
].str.split(",").apply(f)
df = df[~min_len_mask].reset_index(drop=True)
print(
    f"Dropped {min_len_mask.sum()} rows with ligand or receptor chain(s) "
    f"shorter than {minlen}"
)
print(f"Remaining rows: {len(df)}")

Dropped 2118 rows with ligand or receptor chain(s) shorter than 20
Remaining rows: 9930


In [7]:
# check affinity method distribution in the dataset, we will use the Affinity
# Method for handling duplicated sequences
df["Affinity Method"].value_counts(dropna=False)

Affinity Method
NaN        3430
SPR        2431
FL         1090
Other       786
ITC         555
IASP        476
SP          336
RA          320
ELISA       172
IARA        125
BLI         104
Unknown      96
IAGE          9
Name: count, dtype: int64

In [8]:
# replace nan method with "Unknown"
df["Affinity Method"] = df["Affinity Method"].fillna("Unknown")

In [9]:
def filter_duplicates_by_method(
    df,
    seq_col="combined",
    kd_col="KD(M)",
    pkd_col="pKd",
    method_col="Affinity Method",
):
    """
    Group by 'seq_col' (the combined sequence) and resolve duplicates by method priority:
      1. Within each group (all rows sharing the same 'combined' sequence), find the highest-priority method present.
      2. Average the KD(M) and pKd values for all rows of that method.
      3. Keep the *first row* from that subset for all other columns.
    """

    # Priority list of methods to choose from
    method_priority = [
        "SPR",
        "ITC",
        "BLI",
        "RA",
        "FL",
        "SP",
        "ELISA",
        "IASP",
        "IARA",
        "IAGE",
        "Other",
        "Unknown",
    ]

    def choose_best_method(group):
        """
        For a subset of df with the same 'combined' sequence:
          - Iterate over the 'method_priority' list in order.
          - If the group contains that method, average KD(M) across those rows.
          - Return the *first row* of that subset for all other columns.
        """
        methods_in_group = group[method_col].unique()

        for meth in method_priority:
            if meth in methods_in_group:
                # Get only the rows of the current top-priority method
                subset = group[group[method_col] == meth]

                # Average KD(M) and pKD for this subset
                avg_kd = subset[kd_col].mean()
                avg_pkd = subset[pkd_col].mean()

                # Take the *first row* from THIS subset
                # (the subset that contributes to the average)
                first_row = subset.iloc[0].copy()

                # Update the KD(M) and method to reflect the average and the chosen method
                first_row[kd_col] = avg_kd
                first_row[pkd_col] = avg_pkd
                first_row[method_col] = meth

                return first_row
        raise ValueError(
            "None of the methods in the priority list were found in the group!"
        )

    # Apply the 'choose_best_method' to each group of duplicated sequences
    result_df = df.groupby(seq_col, as_index=False).apply(
        choose_best_method, include_groups=False
    )
    return result_df.reset_index(drop=True)

In [10]:
df = filter_duplicates_by_method(df)
print(f"Remaining rows after filtering duplicates: {len(df)}")

8836

In [12]:
# fasta_id will be a unique identifier for each row to be used in fasta files
# instead of complex id as it may contain spaces
df["fasta_id"] = df["PDB"] + "_" + df.index.astype(str)
df.head()

,combined,Source Data Set,Complex ID,PDB,Mutations,Ligand Chains,Receptor Chains,Ligand Name,Receptor Name,KD(M),...,PDB Release Date,Affinity PubMed ID,Affinity Release Date,Subgroup,Ligand Sequences,Receptor Sequences,ligand,receptor,pKd,fasta_id
0,AAAQYPVVNTNYGKIRGLRTPLPNEILGPVEQYLGVPYASPPTGER...,PDBbind v2020,"2WQZ:B, C::PMID=18093521",2WQZ,NaN,B,C,"Neuroligin4, X-Linked",Neurexin-1-beta,1.320000e-07,...,2009-09-08,18093521,2007 Dec 20,NaN,AAAQYPVVNTNYGKIRGLRTPLPNEILGPVEQYLGVPYASPPTGER...,HAGTTYIFSKGGGQITYKWPPNDRPSTRADRLAIGFSTVQKEAVLV...,AAAQYPVVNTNYGKIRGLRTPLPNEILGPVEQYLGVPYASPPTGER...,HAGTTYIFSKGGGQITYKWPPNDRPSTRADRLAIGFSTVQKEAVLV...,6.879426,2WQZ_0
1,AADWDVYCSQDESIPAKFISRLVTSKDQALEKTEINCSNGLVPITQ...,PDBbind v2020,"5IIA:A, B, C, D::PMID=28622512",5IIA,NaN,"B, D","A, C",sperm lysinR,red abalone egg VERL repeat 3 (VR3),1.810000e-09,...,2017-06-14,28622512,2017 Jun 15,NaN,AADWDVYCSQDESIPAKFISRLVTSKDQALEKTEINCSNGLVPITQ...,FLNKAFEVALKVQIIAGFDRGLVKWLRVHGRTLSTVQKKALYFVNR...,AADWDVYCSQDESIPAKFISRLVTSKDQALEKTEINCSNGLVPITQ...,FLNKAFEVALKVQIIAGFDRGLVKWLRVHGRTLSTVQKKALYFVNR...,8.742321,5IIA_1
2,AAEEEDEVEWVVESIAGFLRGPDWSIPILDFVEQKCEVFDDEEESK...,PDBbind v2020,"4ZI2:A, C::PMID=26455799",4ZI2,NaN,C,A,BART-like domain of BARTL1/CCDC104 1-113,Arl3FL-GppNHp,4.200000e-07,...,2015-11-11,26455799,2015 Nov 3,NaN,AAEEEDEVEWVVESIAGFLRGPDWSIPILDFVEQKCEVFDDEEESK...,LLSILRKLKSAPDQEVRILLLGLDNAGKTTLLKQLASEDISHITPT...,AAEEEDEVEWVVESIAGFLRGPDWSIPILDFVEQKCEVFDDEEESK...,LLSILRKLKSAPDQEVRILLLGLDNAGKTTLLKQLASEDISHITPT...,6.376751,4ZI2_2
3,AAIEFDEIVKKLLNIYINDICTTGEKRLLNNYEKSILDRIYKSCEY...,PDBbind v2020,"4UF1:A, B::PMID=26249341",4UF1,NaN,A,B,Bak BH3,Deerpox virus DPV022,6.980000e-06,...,2015-08-05,26249341,2015 Aug,NaN,AAIEFDEIVKKLLNIYINDICTTGEKRLLNNYEKSILDRIYKSCEY...,STMGQVGRQLAIIGDDINRRYD,AAIEFDEIVKKLLNIYINDICTTGEKRLLNNYEKSILDRIYKSCEY...,STMGQVGRQLAIIGDDINRRYD,5.156145,4UF1_3
4,AAILGDEYLWSGGVIPYTFAGVSGADQSAILSGMQELEEKTCIRFV...,PDBbind v2020,"6SAZ:A, B::PMID=31604990",6SAZ,NaN,A,B,Crayfish Astacin,Cleaved human fetuin-b,1.400000e-10,...,2019-10-23,31604990,2019 Oct 11,NaN,AAILGDEYLWSGGVIPYTFAGVSGADQSAILSGMQELEEKTCIRFV...,ALNPSALLSRGCNDSDVLAVAGFALRDINKDRKDGYVLRLNRVNDA...,AAILGDEYLWSGGVIPYTFAGVSGADQSAILSGMQELEEKTCIRFV...,ALNPSALLSRGCNDSDVLAVAGFALRDINKDRKDGYVLRLNRVNDA...,9.853872,6SAZ_4


In [13]:
os.makedirs("data_files", exist_ok=True)
with open("data_files/all_seqs.fasta", "w") as f:
    for i, row in df.iterrows():
        fasta_id = row["fasta_id"]
        sequence = row["combined"]
        f.write(f">{fasta_id}\n{sequence}\n")

we cluster all the sequnces into as many clusters as it takes such that for each cluster the minimum sequence identity between the representative of the cluster and any other sequence in the cluster is at least 10%. We use coverage of the smaller sequence with coverage of 20%. Such loose parameters are used to prevent ending up with thousands of clusters.

In [14]:
!mmseqs easy-cluster data_files/all_seqs.fasta data_files/cluster tmp --min-seq-id 0.1 --cov-mode 5 -c 0.2 -s 10.0 --remove-tmp-files --max-iterations 50000 --cluster-reassign --cluster-steps 9

easy-cluster data_files/all_seqs.fasta data_files/cluster tmp --min-seq-id 0.1 --cov-mode 5 -c 0.2 -s 10.0 --remove-tmp-files --max-iterations 50000 --cluster-reassign --cluster-steps 9 

MMseqs Version:                     	f5f780acd64482cd59b46eae0a107f763cd17b4d
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	10
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.2
Coverage mode                       	5
Compositional bias                  	1
Compositional bias                 

In [16]:
cluster_df = pd.read_csv(
    "data_files/cluster_cluster.tsv", sep="\t", header=None
)
cluster_df = cluster_df.rename(columns={0: "cluster_rep_id", 1: "fasta_id"})
cluster_df

,cluster_rep_id,fasta_id
0,5IIA_1,5IIA_1
1,4ETP_9,4ETP_9
2,4ETP_9,3DCO_7085
3,6IMF_26,6IMF_26
4,4EQA_30,4EQA_30
...,...,...
8831,4IOP_8804,1TDQ_3878
8832,4IOP_8804,3WWK_4913
8833,4IOP_8804,3WWK_4914
8834,4IOP_8804,1IJK_6077


In [17]:
cluster_df_count = cluster_df.groupby(["cluster_rep_id"]).count().sort_values(
    "fasta_id", ascending=False
).rename(columns={"fasta_id": "cluster_size"})
cluster_df_count

,cluster_size
cluster_rep_id,
1FLE_8140,607
2FU5_4863,380
3WQB_7929,334
3QHY_8106,318
1SGD_3901,298
...,...
4C7N_590,1
4C5G_106,1
4BKX_7155,1


In [23]:
train_fasta_ids = []
# Ultimatley, we want 80% of the data to be in the training set, but we know
# that we will need to remove some sequences from test to training later to
# ensure that the training set is not too similar to the test set. So we will
# aim for 60% in the training set for now.
target_train_count = round(0.6 * len(df))

for cluster_rep_id, cluster_size in cluster_df_count.iterrows():
    # add largest cluster first to the training set
    train_fasta_ids.extend(
        cluster_df[cluster_df["cluster_rep_id"] == cluster_rep_id][
            "fasta_id"
        ].tolist()
    )
    if len(train_fasta_ids) >= target_train_count:
        break

df["split"] = df["fasta_id"].apply(
    lambda x: "train" if x in train_fasta_ids else "test"
)
(df["split"].value_counts() / len(df)).round(2)

split
train    0.6
test     0.4
Name: count, dtype: float64

At this point the train and test splits should be slightly dissimilar to each other. But, we want to guarantee minimal similarity, so we will perform a series of iterations to ensure that the splits are as dissimilar as possible. In each iteration we do the following:
1. Compute the pairwise sequence identity between the train and test splits
2. Any sequence in the test split that has a sequence identity of at least 30% with any sequence in the train split and the coverage of the smaller sequence is at least 80%, is moved from the test split to the train split to prevent data leakage.
3. Repeat from step 1 until no sequence is moved from the test split to the train split.

In [24]:
from subprocess import run


def do_split_iteration(df, splits):
    split1, split2 = splits
    with open(f"data_files/{split1}.fasta", "w") as f:
        for _, row in df[df["split"] == split1].iterrows():
            fasta_id = row["fasta_id"]
            seq = row["combined"]
            f.write(f">{fasta_id}\n{seq}\n")

    with open(f"data_files/{split2}.fasta", "w") as f:
        for _, row in df[df["split"] == split2].iterrows():
            fasta_id = row["fasta_id"]
            seq = row["combined"]
            f.write(f">{fasta_id}\n{seq}\n")

    command = [
        "mmseqs",
        "easy-search",
        f"data_files/{split1}.fasta",
        f"data_files/{split2}.fasta",
        "data_files/matches.tsv",
        "tmp",
        "-s",
        "7.5",
        "--min-seq-id",
        "0.3",
        "--cov-mode",
        "5",
        "-c",
        "0.8",
    ]

    result = run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print(result.stderr)
        raise ValueError("Error running mmseqs")
    try:
        matches_df = pd.read_csv(
            "data_files/matches.tsv", sep="\t", header=None
        )
    except pd.errors.EmptyDataError:
        print("No matches found")
        return df, 0
    matches_df = matches_df.rename(
        columns={0: f"{split1}_id", 1: f"{split2}_id", 2: "pident"}
    )
    fasta_ids_to_change_split = matches_df[f"{split2}_id"].unique()
    print(
        f"Changing {len(fasta_ids_to_change_split)} sequences from {split2} "
        f"to {split1}"
    )
    df["split"] = df.apply(
        lambda row: (
            split1
            if row["fasta_id"] in fasta_ids_to_change_split
            else row["split"]
        ),
        axis=1,
    )
    return df, len(fasta_ids_to_change_split)

In [25]:
while True:
    df, total_changed = do_split_iteration(df, ("train", "test"))
    print(
        f"Current split ratios: {(df['split'].value_counts() / len(df)).round(2)}"
    )
    print("*" * 80)
    if (df["split"] == "test").sum() == 0 or total_changed == 0:
        break

Changing 1256 sequences from test to train
Current split ratios: split
train    0.74
test     0.26
Name: count, dtype: float64
********************************************************************************
Changing 315 sequences from test to train
Current split ratios: split
train    0.78
test     0.22
Name: count, dtype: float64
********************************************************************************
Changing 96 sequences from test to train
Current split ratios: split
train    0.79
test     0.21
Name: count, dtype: float64
********************************************************************************
Changing 38 sequences from test to train
Current split ratios: split
train    0.79
test     0.21
Name: count, dtype: float64
********************************************************************************
Changing 8 sequences from test to train
Current split ratios: split
train    0.8
test     0.2
Name: count, dtype: float64
***************************************************

We want to further split the test split into a validation and test split. We will use the same procedure and parameters as above to ensure that the validation and test splits are as dissimilar as possible.

In [39]:
with open("data_files/all_test_seqs.fasta", "w") as f:
    for i, row in df[df["split"] == "test"].iterrows():
        fasta_id = row["fasta_id"]
        sequence = row["combined"]
        f.write(f">{fasta_id}\n{sequence}\n")

In [40]:
!mmseqs easy-cluster data_files/all_test_seqs.fasta data_files/cluster tmp --min-seq-id 0.1 --cov-mode 5 -c 0.2 -s 10.0 --remove-tmp-files --max-iterations 50000 --cluster-reassign --cluster-steps 9

easy-cluster data_files/all_test_seqs.fasta data_files/cluster tmp --min-seq-id 0.1 --cov-mode 5 -c 0.2 -s 10.0 --remove-tmp-files --max-iterations 50000 --cluster-reassign --cluster-steps 9 

MMseqs Version:                     	f5f780acd64482cd59b46eae0a107f763cd17b4d
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	10
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.2
Coverage mode                       	5
Compositional bias                  	1
Compositional bias            

In [41]:
cluster_df = pd.read_csv(
    "data_files/cluster_cluster.tsv", sep="\t", header=None
)
cluster_df = cluster_df.rename(columns={0: "cluster_rep_id", 1: "fasta_id"})
cluster_df_count = (
    cluster_df.groupby(["cluster_rep_id"])
    .count()
    .sort_values("fasta_id", ascending=False)
    .rename(columns={"fasta_id": "cluster_size"})
)
cluster_df_count

,cluster_size
cluster_rep_id,
5M2O_5685,40
1IAR_3446,37
4PWX_8349,34
3MA2_796,34
1FCC_6364,33
...,...
4BKX_7155,1
4C5G_106,1
4C7N_590,1


In [42]:
# evenly distribute the clusters between val and test
val_fasta_ids = []
add_to_val = True
for cluster_rep_id, cluster_size in cluster_df_count.iterrows():
    if add_to_val:
        val_fasta_ids.extend(
            cluster_df[cluster_df["cluster_rep_id"] == cluster_rep_id][
                "fasta_id"
            ].tolist()
        )
    add_to_val = not add_to_val

df["split"] = df.apply(
    lambda row: "val" if row["fasta_id"] in val_fasta_ids else row["split"],
    axis=1,
)
(df["split"].value_counts() / len(df)).round(2)

split
train    0.8
val      0.1
test     0.1
Name: count, dtype: float64

In [ ]:
# perform final split iterations to ensure that the val set is not too similar
# to the test set
while True:
    df, total_changed = do_split_iteration(df, ("val", "test"))
    print(
        f"Current split ratios: {(df['split'].value_counts() / len(df)).round(2)}"
    )
    print("*" * 80)
    if (df["split"] == "test").sum() == 0 or total_changed == 0:
        break

df["split"].value_counts()

Changing 54 sequences from test to val
Current split ratios: split
train    0.80
val      0.11
test     0.09
Name: count, dtype: float64
********************************************************************************
Changing 16 sequences from test to val
Current split ratios: split
train    0.80
val      0.11
test     0.09
Name: count, dtype: float64
********************************************************************************
Changing 6 sequences from test to val
Current split ratios: split
train    0.80
val      0.11
test     0.09
Name: count, dtype: float64
********************************************************************************
Changing 1 sequences from test to val
Current split ratios: split
train    0.80
val      0.11
test     0.09
Name: count, dtype: float64
********************************************************************************
No matches found
Current split ratios: split
train    0.80
val      0.11
test     0.09
Name: count, dtype: float64
***************

split
train    7028
val       993
test      815
Name: count, dtype: int64

In [44]:
df = df.drop(columns=["ligand", "receptor", "combined", "fasta_id"])
df.to_csv("PPB_Affinity_processed_filtered.csv", index=False)